<a href="https://colab.research.google.com/github/matHoYoung/AI_18_JHY_section1/blob/main/AI_18_%EC%A0%95%ED%98%B8%EC%98%81_section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name : 게임의 이름입니다.

Platform : 게임이 지원되는 플랫폼의 이름입니다.

Year : 게임이 출시된 연도입니다.

Genre : 게임의 장르입니다.

Publisher : 게임을 배급한 회사입니다.

NA_Sales : 북미지역에서의 출고량입니다.

EU_Sales : 유럽지역에서의 출고량입니다.

JP_Sales : 일본지역에서의 출고량입니다.

Other_Sales : 기타지역에서의 출고량입니다.



# 필수 조건

##다음 필수 조건은 전부 만족해야 2점을 받을 수 있습니다.

- 영상 길이 조건을 만족했는가?
 영상이 5분 ~ 10분 사이로 제작됨

- 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답했는가?

 -지역은 어떻게 구분했는지 설명함

 -지역별로 선호하는 게임의 장르를 파악함

 -지역별로 선호하는 게임의 장르가 왜 같다고/다르다고 생각했는지를 설명함

- 연도별 게임의 트렌드가 있을까 라는 질문에 대답했는가?

 -트렌드는 어떤 것을 기준으로 파악했는지 설명함

 -연도별로 트렌드가 존재하는지 여부를 언급함

 -왜 존재한다/존재하지 않는다고 생각했는지 설명함

- 인기가 많은 게임에 대한 분석 및 시각화 프로세스가 포함되었는가?

 -인기가 많다는 것을 어떻게 정의했는지 설명함

 -해당 게임이 왜 인기가 많다고 생각했는지 설명함

 -시각화를 통해 무엇을 나타내고자 하는지를 고려하여 파라미터를 조정함 (제목, 축이름 필수, 그 외 파라미터는 선택사항)

 -해당 분석을 통해 어떤 인사이트를 도출할 수 있는지 설명함

- 다음 분기에 어떤 게임을 설계해야 하는지에 대한 결론이 제시되었는가?
 분석에 대한 결론이 영상에서 제시됨

# 추가 조건 (Advanced Goal)
## 다음 추가 조건은 1개 이상 만족 시 3점을 받을 수 있습니다.

-디테일한 시점 및 목표 설정이 문제 정의 과정에서 제시되었는가?

-도메인 지식을 활용하여 비지니스 인사이트를 도출하였는가?

-제공된 데이터셋 외에 추가 데이터를 사용하였는가?

-통계적 기법을 활용한 가설 검정을 진행하였는가?

-완성도 높은 시각화를 보여주었는가?

-가독성, 목적성이 잘 드러나는 시각화

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving vgames2.csv to vgames2.csv


In [ ]:
game = pd.read_csv('vgames2.csv',index_col = 0)
game.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [ ]:
game.shape

(16598, 9)

In [ ]:
# 품질적 문제와 구조적 문제 확인하기. 

In [ ]:
# 품질적 문제 확인 
#중복값, 결측치, 부정확한 데이터 있는지 파악해보자. 

In [ ]:
#중복값은 1이므로 분석 결과에 큰 영향을 끼치지 않는 다고 1개의 중복 값 제거
game.duplicated().sum()
game = game.drop_duplicates()

1

In [ ]:
# index 재정렬
game = game.reset_index(drop=True)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [ ]:
#각 컬럼별 결측치 확인하기. 
# Name 결측치 0 
game[game['Name'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


In [ ]:
# Platform 결측치 0 
game[game['Platform'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


In [ ]:
# Year 결측치 270 
game[game['Year'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
273,Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
358,Majesty 2: The Fantasy Kingdom Sim,X360,NaN,Simulation,Unknown,0.03,0,0,0
430,Famista 64,N64,NaN,Sports,Namco Bandai Games,0,0,0.17,0.03
...,...,...,...,...,...,...,...,...,...
16239,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0,0,0
16274,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0M,0.03
16277,Madden NFL 11,Wii,NaN,Sports,Unknown,0.7,0,0,50K
16424,Yoostar on MTV,X360,NaN,Misc,Unknown,0.06,0,0,0
